In [1]:
%matplotlib inline

import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from scipy.stats import linregress
from scipy.stats import norm
from operator import itemgetter
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn import utils

In [2]:
dfTheseTeams = pd.read_csv('final-csv-data/NFL-Curr-Teams.csv')

dfOldTeams = pd.read_csv('final-csv-data/NFL-Prev-Teams.csv') #for cause of change in W/L look at NYPA,NYPF,NYYdsA,NYEXP stuff,NYoPtspG, NYSRS stuff, NYTotYds, NYRZPct, NYoRZPct,NYQbDYAR,NYAdjOLineYards,NYAdjDLineYards,NYOFFDVOA stuff,NYDEFDVOA stuff,NYSBWin
dfOldTeams.head()

,Team,PA,YdsA,PlyA,pYPPa,TOa,FLa,T1stDa,PCmpa,PAtta,...,Unnamed: 529,NYDraftVal,NYPrevAV,NYCurrAV,NYAVChange,NYSBOdds,NY2YrWins,NY2YrAvg,NYQBAdded,NYSBWin
0,Cardinals,362,4883,1019,4.8,28,14,293,344,547,...,NaN,40.5,212,188,-24,10000,7,7.5,1.0,0
1,Falcons,406,5939,1059,5.6,22,10,358,422,655,...,NaN,34.3,249,222,-27,1600,11,10.5,0.0,0
2,Ravens,321,5154,993,5.2,28,10,294,369,578,...,NaN,54.8,205,221,16,5000,8,8.5,0.5,0
3,Bills,378,5712,1019,5.6,18,6,338,306,511,...,NaN,54.5,205,195,-10,8000,7,8.0,1.0,0
4,Panthers,402,5756,1035,5.6,27,10,329,409,612,...,NaN,40.6,195,218,23,4000,6,8.5,0.0,0


In [3]:
theseTeamsTest = dfTheseTeams.iloc[:, [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,28,31,33,34,35,36,37,38,39,40,42,43,44,49,50,58,59,60,61,62,63,64,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,91,92,93,94,95,96,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,124,127,129,130,131,132,133,134,135,136,137,139,140,141,142,143,147,149,150,151,159,160,161,163,164,165,166,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,190,192,194,195,197,198,199,200,203,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,235,236,237,238,239,240,241,242,246,247,248,250,251,252,253,254,255,256,257,261,262,263,264,265,266,267,268,269]]
oldTeamsTest = dfOldTeams.iloc[:, [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,28,31,33,34,35,36,37,38,39,40,42,43,44,49,50,58,59,60,61,62,63,64,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,91,92,93,94,95,96,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,124,127,129,130,131,132,133,134,135,136,137,139,140,141,142,143,147,149,150,151,159,160,161,163,164,165,166,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,190,192,194,195,197,198,199,200,203,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,235,236,237,238,239,240,241,242,246,247,248,250,251,252,253,254,255,256,257,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,297,300,302,303,304,305,306,307,308,309,311,312,313,318,319,327,328,329,330,331,332,333,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,360,361,362,363,364,365,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,393,396,398,399,400,401,402,403,404,405,406,408,409,410,411,412,416,418,419,420,428,429,430,432,433,434,435,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,459,461,463,464,466,467,468,469,472,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,504,505,506,507,508,509,510,511,515,516,517,518,519,520,521,522,523,524,525,526,530,531,532,533,534,535,536,537,538]]


theseTeamsNames = dfTheseTeams.iloc[:,0]
oldTeamsNames = dfOldTeams.iloc[:,0]

theseTeamsTest.head()

,PA,YdsA,PlyA,pYPPa,TOa,FLa,T1stDa,PCmpa,PAtta,PYdsa,...,VAR,DraftVal,PrevAV,CurrAV,AVChange,SBOdds,2YrWins,2YrAvg,QBAdded,SBWin
0,361,4973,1008,4.9,21,6,283,343,564,3541,...,0.04,40.5,212,188,-24,10000,7,7.5,1.0,0
1,315,5094,994,5.1,16,8,323,362,553,3429,...,0.04,34.3,249,222,-27,1600,11,10.5,0.0,0
2,303,5201,1045,5.0,34,12,296,333,569,3421,...,0.11,54.8,205,221,16,5000,8,8.5,0.5,0
3,359,5682,1066,5.3,25,7,348,370,573,3688,...,0.05,54.5,205,195,-10,8000,7,8.0,1.0,0
4,327,5074,953,5.3,21,11,282,351,550,3665,...,0.05,40.6,195,218,23,4000,6,8.5,0.0,0


In [4]:
colArr = dfOldTeams.iloc[:,[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,28,31,33,34,35,36,37,38,39,40,42,43,44,49,50,58,59,60,61,62,63,64,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,91,92,93,94,95,96,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,124,127,129,130,131,132,133,134,135,136,137,139,140,141,142,143,147,149,150,151,159,160,161,163,164,165,166,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,190,192,194,195,197,198,199,200,203,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,235,236,237,238,239,240,241,242,246,247,248,250,251,252,253,254,255,256,257,261,262,263,264,265,266,267,268,269]]
fyArr = dfOldTeams.iloc[:, [270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,297,300,302,303,304,305,306,307,308,309,311,312,313,318,319,327,328,329,330,331,332,333,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,360,361,362,363,364,365,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,393,396,398,399,400,401,402,403,404,405,406,408,409,410,411,412,416,418,419,420,428,429,430,432,433,434,435,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,459,461,463,464,466,467,468,469,472,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,504,505,506,507,508,509,510,511,515,516,517,518,519,520,521,522,523,524,525,526,530,531,532,533,534,535,536,537,538]]
rSquareArr = []
colUse = []
fyUse = []
for i in colArr:
    for j in fyArr:
        
        #plt.style.use('dark_background')

        #yardsAllRBs, ax = plt.subplots()
        
        slope,intercept, r_value, p_value, std_err = linregress(dfOldTeams[i], dfOldTeams[j])
        rsquaredArr1 = r_value**2
        rSquareArr.append(rsquaredArr1)
        colUse.append(i)
        fyUse.append(j)

        #ax.scatter(dfFYRBs[i], dfFYRBs[j], color = 'orange', label = "Running backs")
        #ax.axvline(x = np.mean(dfFYRBs[i]), color = 'white')
        #ax.axhline(y = np.mean(dfFYRBs[j]), label = "Average", color = 'white')
        #yardsAllRBs.suptitle("Correlation between "+i+" and "+j, weight = 'bold', size = 18, y = 1.05)
        #ax.set_xlabel(i)
        #ax.set_ylabel(j)

        #ax.plot(np.unique(dfFYRBs[i]), np.poly1d(np.polyfit(dfFYRBs[i], dfFYRBs[j],1))(np.unique(dfFYRBs[i])))

        #ax.legend(loc='best', prop={'size':9, "family":"DejaVu Sans"})
        #rpString = "r = " + str(round(r_value,3)) + ", p = " + str(round(p_value,3))+ ", rsquared = " + str(round(rsquaredArr1, 3))

        #ax.set_title("%s"%rpString, size = 12, fontname = 'DejaVu Sans')
        #yardsAllRBs.savefig('newGraphs/CollegetoNFL-correlation'+i+'to'+j+'.png', dpi = 400, bbox_inches = 'tight')
#for n,m,f in zip(rSquareArr,colUse,fyUse):
 #   print(n,m,f)


C:\Users\Kids\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\Kids\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\Kids\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [5]:
#plt.style.use('seaborn-poster')

#compareStats, ax = plt.subplots()

statsGraph = []
for i in rSquareArr:
    statsGraph.append(float(i))
combinedStats = [[i,j,k] for i,j,k in zip(statsGraph,colUse,fyUse)]


sortedRSquared = sorted(combinedStats, key = itemgetter(0))
#print(sortedRSquared)

sortedRSquaredData = [row[0] for row in sortedRSquared]

In [182]:
train, test = train_test_split(dfOldTeams, test_size = 0.25, random_state = 99)

xtrainRB = train[['PA','YdsA','PlyA','pYPPa','TOa','FLa','T1stDa','PCmpa','PAtta','PYdsa','PTDa','Inta','NYPAa','p1stDa','Atta','RYdsa','TDa','YPAa','1stDa','Pena','PenYdsa','1stPya','ScPerca','TOPerca','dEXP','CmpPerca','TDPerca','IntPerca','pYpAa','AYpAa','YpCa','YpGa','Ratea','Skf','SkYds','ANYpAa','SkPercf','pdEXP','rYpGa','rdEXP','oAllTD','o2PM','o2PA','oXPM','oXPA','oFGM','oFGA','oPtspG','o3DAtt','o3DConv','o3DPerc','o4DAtt','o4DConv','o4DPerc','oRZAtt','oRZScore','oRZPct','oNumDr','oPlays','oScPerc','oTOPerc','oPlaysPD','oYdsPD','oStart','oTimePD','oPtsPD','W','L','WLPerc','PF','PD','MoV','SoS','SRS','OSRS','DSRS','TotYds','Ply','YpP','TO','FL','Pa1stD','PCmp','PAtt','PYds','PTD','Int','NYpA','p1stD','Att','rYds','TD','YpA','1stD','Pen','PenYds','1stPy','ScoPerc','TORPerc','EXP','CmpPerc','TDPerc','IntPerc','pLng','pYpA','pAYpA','YpC','pYpG','Rate','Sk','SkYdsa','pANYpA','SkPerc','4QC','GWD','pEXP','Lng','YpG','Fmb','rEXP','AllTD','2PaM','2PA','XPM','XPA','FGM','FGA','PtspG','3DAtt','3DConv','3DPerc','4DAtt','4DConv','4DPerc','RZAtt','RZScore','RZPct','NumDr','Plays','ScPerc','TOPerc','PlaysPD','YdsPD','Start','Time','Pts','QbDYAR','QbYAR','QbDVOA','QbVOA','QBR','QbPass','QbYards','EYds','QbTD','QbINT','ALEX','AdjOLineYards','RBYards','PowerSuccess','PowerRank','Stuffed','StuffedRank','2ndLevelYards','2ndLevelRank','OpenFieldYards','OpenFieldRank','Sacks','AdjustedSackRate','AdjDLineYards','dRBYards','dPowerSuccess','dPowerRank','dStuffed','dStuffedRank','d2ndLevelYards','d2ndLevelRank','dOpenFieldYards','dOpenFieldRank','dSacks','dAdjustedSackRate','OFFDVOA','LASTYEARODVOA','WEIOFFENSE','PASSOFF','PASSRANK','RUSHOFF','RUSHRANK','TOTALOFF','PASSDVOA','RUSHDVOA','oVAR','DEFDVOA','LASTYEARDEFDVOA','WEIDEFENSE','PASSDEF','PASSDEFRANK','RUSHDEF','RUSHDEFRANK','TOTALDEF','PASSDEFWEI','RUSHDEFWEI','VAR','DraftVal','PrevAV','CurrAV','AVChange','SBOdds','2YrWins','2YrAvg','QBAdded','SBWin']]
ytrainRB = train[['NYoTOPerc']].astype(int)

xtestRB = test[['PA','YdsA','PlyA','pYPPa','TOa','FLa','T1stDa','PCmpa','PAtta','PYdsa','PTDa','Inta','NYPAa','p1stDa','Atta','RYdsa','TDa','YPAa','1stDa','Pena','PenYdsa','1stPya','ScPerca','TOPerca','dEXP','CmpPerca','TDPerca','IntPerca','pYpAa','AYpAa','YpCa','YpGa','Ratea','Skf','SkYds','ANYpAa','SkPercf','pdEXP','rYpGa','rdEXP','oAllTD','o2PM','o2PA','oXPM','oXPA','oFGM','oFGA','oPtspG','o3DAtt','o3DConv','o3DPerc','o4DAtt','o4DConv','o4DPerc','oRZAtt','oRZScore','oRZPct','oNumDr','oPlays','oScPerc','oTOPerc','oPlaysPD','oYdsPD','oStart','oTimePD','oPtsPD','W','L','WLPerc','PF','PD','MoV','SoS','SRS','OSRS','DSRS','TotYds','Ply','YpP','TO','FL','Pa1stD','PCmp','PAtt','PYds','PTD','Int','NYpA','p1stD','Att','rYds','TD','YpA','1stD','Pen','PenYds','1stPy','ScoPerc','TORPerc','EXP','CmpPerc','TDPerc','IntPerc','pLng','pYpA','pAYpA','YpC','pYpG','Rate','Sk','SkYdsa','pANYpA','SkPerc','4QC','GWD','pEXP','Lng','YpG','Fmb','rEXP','AllTD','2PaM','2PA','XPM','XPA','FGM','FGA','PtspG','3DAtt','3DConv','3DPerc','4DAtt','4DConv','4DPerc','RZAtt','RZScore','RZPct','NumDr','Plays','ScPerc','TOPerc','PlaysPD','YdsPD','Start','Time','Pts','QbDYAR','QbYAR','QbDVOA','QbVOA','QBR','QbPass','QbYards','EYds','QbTD','QbINT','ALEX','AdjOLineYards','RBYards','PowerSuccess','PowerRank','Stuffed','StuffedRank','2ndLevelYards','2ndLevelRank','OpenFieldYards','OpenFieldRank','Sacks','AdjustedSackRate','AdjDLineYards','dRBYards','dPowerSuccess','dPowerRank','dStuffed','dStuffedRank','d2ndLevelYards','d2ndLevelRank','dOpenFieldYards','dOpenFieldRank','dSacks','dAdjustedSackRate','OFFDVOA','LASTYEARODVOA','WEIOFFENSE','PASSOFF','PASSRANK','RUSHOFF','RUSHRANK','TOTALOFF','PASSDVOA','RUSHDVOA','oVAR','DEFDVOA','LASTYEARDEFDVOA','WEIDEFENSE','PASSDEF','PASSDEFRANK','RUSHDEF','RUSHDEFRANK','TOTALDEF','PASSDEFWEI','RUSHDEFWEI','VAR','DraftVal','PrevAV','CurrAV','AVChange','SBOdds','2YrWins','2YrAvg','QBAdded','SBWin']]
ytestRB = test[['NYoTOPerc']].astype(int)

#knn = KNeighborsClassifier(n_neighbors=5)
#knn.fit(xtrainRB,ytrainRB)
#knn.score(xtestRB,ytestRB)
#print(knn.kneighbors(xtrain, return_distance=False))

In [183]:
linReg = linear_model.LinearRegression(fit_intercept = False)
linReg.fit(xtrainRB,ytrainRB)

y_predLinRB = linReg.predict(xtestRB)
linR2RB = abs(1/(r2_score(ytestRB,y_predLinRB)))
print("Mean squared error: %.3f"%mean_squared_error(ytestRB, y_predLinRB))
print("Variance score: %.3f"%r2_score(ytestRB,y_predLinRB))

cvScoreLin = cross_val_score(linReg,xtestRB,ytestRB,cv=4,scoring='explained_variance')
print("Accuracy (cross validation score): %0.2f (+/- %0.2f)"%(cvScoreLin.mean(), cvScoreLin.std()*2))
print(linR2RB)

Mean squared error: 30.223
Variance score: -2.539
Accuracy (cross validation score): -4.28 (+/- 5.87)
0.39385774213751323


In [184]:
ridgeReg = linear_model.Ridge(alpha=5, fit_intercept = False)
ridgeReg.fit(xtrainRB,ytrainRB)

y_predRidgeRB = ridgeReg.predict(xtestRB)
ridgeR2RB = abs(1/(r2_score(ytestRB,y_predRidgeRB)))
print("Mean squared error: %.3f"%mean_squared_error(ytestRB, y_predRidgeRB))
print("Variance score: %.3f"%r2_score(ytestRB,y_predRidgeRB))

cvScoreRidge = cross_val_score(ridgeReg,xtestRB,ytestRB,cv=4,scoring='explained_variance')
print("Accuracy (cross validation score): %0.2f (+/- %0.2f)"%(cvScoreRidge.mean(), cvScoreRidge.std()*2))
print(ridgeR2RB)

Mean squared error: 29.789
Variance score: -2.488
Accuracy (cross validation score): -4.27 (+/- 5.87)
0.40188937928449825


In [185]:
lassoReg = linear_model.Lasso(alpha = 5, tol = 0.1, max_iter = 100000, fit_intercept = False)
lassoReg.fit(xtrainRB,ytrainRB)

y_predLassoRB = lassoReg.predict(xtestRB)
lassoR2RB = abs(1/(r2_score(ytestRB,y_predLassoRB)))
print("Mean squared error: %.3f"%mean_squared_error(ytestRB, y_predLassoRB))
print("Variance score: %.3f"%r2_score(ytestRB,y_predLassoRB))

cvScoreLasso = cross_val_score(lassoReg,xtestRB,ytestRB,cv=4,scoring='explained_variance')
print("Accuracy (cross validation score): %0.2f (+/- %0.2f)"%(cvScoreLasso.mean(), cvScoreLasso.std()*2))

print(lassoR2RB)

Mean squared error: 9.126
Variance score: -0.069
Accuracy (cross validation score): -0.79 (+/- 1.08)
14.57780022690138


In [186]:
logReg = linear_model.LogisticRegression(fit_intercept = False, dual=False)
logReg.fit(xtrainRB,ytrainRB)

y_predLogRB = logReg.predict(xtestRB)

print("Mean squared error: %.3f"%mean_squared_error(ytestRB, y_predLogRB))
print("Variance score: %.3f"%r2_score(ytestRB,y_predLogRB))

Mean squared error: 23.917
Variance score: -1.801


C:\Users\Kids\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [187]:
linear_rbs = linReg.predict(theseTeamsTest)
for i,j in zip(linear_rbs, theseTeamsNames):
    print(i,j)

[1.2948129] Cardinals
[15.4103737] Falcons
[10.12674038] Ravens
[16.75414918] Bills
[7.9762261] Panthers
[11.1500005] Bears
[14.89465607] Bengals
[9.848471] Browns
[22.48888671] Cowboys
[4.37892181] Broncos
[14.76575799] Lions
[8.90328731] Packers
[13.32320136] Texans
[9.08671843] Colts
[15.81620735] Jaguars
[11.46743161] Chiefs
[7.56338096] Chargers
[11.74558006] Rams
[7.10111015] Dolphins
[6.37866404] Vikings
[17.43150093] Patriots
[18.03754533] Saints
[14.87999068] Giants
[14.84117326] Jets
[14.8018414] Raiders
[3.93068446] Eagles
[16.43705477] Steelers
[11.91272538] 49ers
[3.35022126] Seahawks
[5.95017067] Buccaneers
[16.34975254] Titans
[11.22770018] Redskins


In [188]:
ridge_rbs = ridgeReg.predict(theseTeamsTest)
for i,j in zip(ridge_rbs, theseTeamsNames):
    print(i,j)

[1.5206989] Cardinals
[15.31773514] Falcons
[10.0832649] Ravens
[16.61361576] Bills
[8.02501339] Panthers
[10.98999321] Bears
[14.77915062] Bengals
[9.85642668] Browns
[22.30636621] Cowboys
[4.37831925] Broncos
[14.66463425] Lions
[9.03183824] Packers
[13.34067099] Texans
[9.31892464] Colts
[15.719687] Jaguars
[11.57543508] Chiefs
[7.52354475] Chargers
[11.79721576] Rams
[7.06212331] Dolphins
[6.41442196] Vikings
[17.33808401] Patriots
[17.80007815] Saints
[14.87268456] Giants
[14.7465433] Jets
[14.60566539] Raiders
[3.88666106] Eagles
[16.23789786] Steelers
[12.07714368] 49ers
[3.51918065] Seahawks
[5.97842655] Buccaneers
[16.43446895] Titans
[11.19119068] Redskins


In [189]:
lasso_rbs = lassoReg.predict(theseTeamsTest)
for i,j in zip(lasso_rbs, theseTeamsNames):
    print(i,j)

10.892748680800995 Cardinals
9.667213911186142 Falcons
12.372971192580462 Ravens
10.595603943236444 Bills
11.194125457332177 Panthers
9.432635648736639 Bears
10.215377672936048 Bengals
11.116024958054878 Browns
10.664141852747345 Cowboys
12.236551925976926 Broncos
11.457005335955346 Lions
11.083900278479241 Packers
12.698178037672223 Texans
10.3827512282733 Colts
12.198916071283685 Jaguars
10.333557418257108 Chiefs
9.755753705643672 Chargers
12.31861681250864 Rams
10.367047688835738 Dolphins
10.017199197381073 Vikings
9.180895338826947 Patriots
10.779831606970221 Saints
10.969084121149022 Giants
10.231818859383164 Jets
10.218709451724425 Raiders
11.29963808640608 Eagles
10.438025565214032 Steelers
10.900737227911812 49ers
11.21567763762213 Seahawks
9.292897679730562 Buccaneers
11.040591561986162 Titans
11.565723451811657 Redskins


In [190]:
log_rbs = logReg.predict(theseTeamsTest)
for i,j in zip(log_rbs, theseTeamsNames):
    print(i,j)

9 Cardinals
11 Falcons
7 Ravens
13 Bills
7 Panthers
6 Bears
8 Bengals
10 Browns
15 Cowboys
6 Broncos
12 Lions
8 Packers
13 Texans
13 Colts
7 Jaguars
13 Chiefs
15 Chargers
14 Rams
11 Dolphins
7 Vikings
11 Patriots
15 Saints
13 Giants
10 Jets
13 Raiders
10 Eagles
11 Steelers
8 49ers
6 Seahawks
12 Buccaneers
16 Titans
8 Redskins


In [191]:
averageProd = []
for i,j,h in zip(linear_rbs,ridge_rbs,lasso_rbs):
    #averageProd.append(float((i*linR2RB+j*ridgeR2RB+h*lassoR2RB)/(linR2RB+ridgeR2RB+lassoR2RB)))
    averageProd.append(float((i+j+h)/3))

combinedAvg = [[i,j] for i,j in zip(theseTeamsNames, averageProd)]

sortedAvg = sorted(combinedAvg, key = itemgetter(1))
print(sortedAvg)

#this average is raw talent average, got rid of 16,17 in FYRB struct (draftpos and dcpos)

[['Cardinals', 4.569420161555077], ['Seahawks', 6.028359849203355], ['Eagles', 6.372327870298868], ['Broncos', 6.997930993994242], ['Buccaneers', 7.073831632683856], ['Vikings', 7.603428397006321], ['Dolphins', 8.176760383305698], ['Chargers', 8.28089313604449], ['Panthers', 9.065121648008402], ['Colts', 9.596131433076934], ['Packers', 9.673008611451822], ['Browns', 10.273640879342226], ['Bears', 10.52420978525293], ['Ravens', 10.860992159481802], ['Chiefs', 11.125474702894131], ['Redskins', 11.32820477247238], ['49ers', 11.63020209383292], ['Rams', 11.953804212195621], ['Texans', 13.120683463149353], ['Raiders', 13.208738748156541], ['Jets', 13.273178472138936], ['Bengals', 13.296394787206888], ['Falcons', 13.465107582767914], ['Giants', 13.573919784693013], ['Lions', 13.629132525831515], ['Steelers', 14.370992732547306], ['Jaguars', 14.578270140960656], ['Titans', 14.608271016898273], ['Patriots', 14.650160094333089], ['Bills', 14.65445629494567], ['Saints', 15.539151695922476], ['Co